In [453]:
import numpy as np
import kwant

In [454]:
num_vectors = 3
dim = 10
ham = kwant.rmt.gaussian(dim)

In [455]:
vectors = np.identity(dim)[:num_vectors]

In [456]:
eigenvalues = np.diagonal(ham)[:num_vectors]

In [457]:
eigenvalues

array([-0.86100547+0.j,  1.10731596+0.j, -1.72619685+0.j])

In [458]:
from kpm_funcs import _kernel, _make_vector_factory

In [459]:
def build_greens_function(ham, params=None, vectors=None, kpm_params=dict()):
    """Build a Green's function operator.

    Returns a function that takes a Fermi energy, and returns the
    Green's function of the `vectors` over the occupied energies of the
    Hamiltonian.

    Parameters
    ----------
    ham : kwant.System or ndarray
        Finalized kwant system or ndarray of the Hamiltonian.
    params : dict, optional
        Parameters for the kwant system.
    vectors : ndarray (M,N), optional
        Vectors upon which the projector will act. `M` is the
        number of vectors and `N` the number of orbitals in the
        system.
    kpm_params : dict, optional
        Dictionary containing the parameters to pass to the `~kwant.kpm`
        module. 'num_vectors' will be overwritten to match the number
        of vectors, and 'operator' key will be deleted.
    """
    if vectors is None:
        kpm_params['vector_factory'] = None
    else:
        vectors = np.atleast_2d(vectors)
        num_vectors = vectors.shape[0]
        kpm_params['num_vectors'] = num_vectors
        kpm_params['vector_factory'] = _make_vector_factory(vectors)

    # extract the number of moments or set default to 100
    num_moments = kpm_params.get('num_moments', 100)
    # prefactors of the kernel in kpm
    m = np.arange(num_moments)
    gs = _kernel(np.ones(num_moments))
    gs[0] = gs[0] / 2

    kpm_params['num_moments'] = num_moments
    # overwrite operator to extract kpm expanded vectors only
    kpm_params['operator'] = lambda bra, ket: ket

    # calculate kpm expanded vectors
    spectrum = kwant.kpm.SpectralDensity(ham, params=params, **kpm_params)
    print('Bounds of initial hamiltonian', spectrum.bounds)
    expanded_vectors = np.array(spectrum._moments_list)


    def get_coefs(e_F):
        # TODO put the proper coefficients here
        e_F = np.atleast_1d(e_F)
        e_rescaled = (e_F - spectrum._b) / spectrum._a
        phi_e = np.arccos(e_rescaled)
        prefactor = 2j / np.sqrt(1 - e_rescaled)
        coef = gs * np.exp(-1j * np.outer(phi_e, m))
        coef = prefactor[:, None] * coef
        return coef

    return lambda e_F : get_coefs(e_F) @ expanded_vectors.T

In [460]:
params = None
kpm_params = dict()

if vectors is None:
    kpm_params['vector_factory'] = None
else:
    num_vectors = vectors.shape[0]
    kpm_params['num_vectors'] = num_vectors
    kpm_params['vector_factory'] = _make_vector_factory(vectors)

# extract the number of moments or set default to 100
num_moments = kpm_params.get('num_moments', 100)
kpm_params['num_moments'] = num_moments
# overwrite operator to extract kpm expanded vectors only
kpm_params['operator'] = lambda bra, ket: ket

# calculate kpm expanded vectors
spectrum = kwant.kpm.SpectralDensity(ham, params=params, **kpm_params)
expanded_vectors = np.array(spectrum._moments_list)

In [461]:
expanded_vectors.shape

(3, 100, 10)

In [462]:
e_F = [0, 1]
# TODO put the proper coefficients here
e_F = np.atleast_1d(e_F)
e_rescaled = (e_F - spectrum._b) / spectrum._a
phi_e = np.arccos(e_rescaled)
prefactor = 2j / np.sqrt(1 - e_rescaled)
m = np.arange(num_moments)
gs = _kernel(np.ones(num_moments))

In [463]:
e_F.shape, e_rescaled.shape, phi_e.shape, prefactor.shape, m.shape, gs.shape

((2,), (2,), (2,), (2,), (100,), (100,))

In [464]:
np.outer(m, phi_e).shape

(100, 2)

In [465]:
(gs * np.outer(phi_e, m)).shape

(2, 100)

In [466]:
(prefactor[:,None] * (gs * np.outer(phi_e, m))).shape

(2, 100)

In [467]:
coef = (prefactor[:,None] * (gs * np.outer(phi_e, m)))
coef.shape

(2, 100)

In [468]:
expanded_vectors.shape

(3, 100, 10)

In [469]:
(expanded_vectors.T).shape

(10, 100, 3)

In [470]:
(coef @ expanded_vectors.T).shape

(10, 2, 3)

In [471]:
greens = (coef @ expanded_vectors.T)

In [472]:
g = build_greens_function(ham, vectors=vectors)

In [473]:
g([0,1]).shape

(10, 2, 3)

#### now build the perturbation matrix

In [474]:
psi = g(eigenvalues[:2])

In [475]:
psi.shape

(10, 2, 3)

In [476]:
# we want only the diagonal elements that map each vector to its eigenvalue
psi_i = np.diagonal(psi, axis1=1, axis2=2)

In [477]:
psi_i.shape

(10, 2)

In [478]:
(psi_i.conj().T @ psi_i).shape

(2, 2)

#### alternative way of calculating

In [479]:
greens = lambda vec, e: build_greens_function(ham, vectors=vec)(e).squeeze()

In [480]:
greens(vectors[0], eigenvalues[0]).shape

(10,)

In [481]:
psi_i = np.array([greens(vec, e) for (vec, e) in zip(vectors, eigenvalues)]).T

In [482]:
psi_i.shape

(10, 3)

In [483]:
(psi_i.conj().T @ psi_i).shape

(3, 3)

### perturbation elements of the matrix

In [484]:
import scipy

In [485]:
def build_perturbation(eigenvalues, psi, ham, params=None):
    """Build the perturbation elements `<psi_i|H|psi_j>`.

    Given a perturbed Hamiltonian `H`, we calculate the the
    perturbation approximation of the effect of the complement
    space `B` on the space `A`.
    The vectors `psi` expand a space `A`, which complement is `B`.
    `psi` are eigenvectors of `H_0` (not specified) with a
    corresponding set of `eigenvalues`.

    Parameters
    ----------
    eigenvalues : (M) array of floats
        Eigenvalues of the eigenvectors `psi` of `H_0`.
    psi : (M, N) ndarray
        Vectors of length (N), the same as the system defined
        by `ham`.
    ham : kwant.System or ndarray
        Finalized kwant system or ndarray of the Hamiltonian.
    params : dict, optional
        Parameters for the kwant system.

    Returns
    -------
    ham_ij : (M, M) ndarray
        Matrix elements of the second order perturbation
        of subspace `A` due to the interaction `H` with
        the subspace `B`.
    """
    # Normalize the format of vectors
    eigenvalues = np.atleast_1d(eigenvalues)
    psi = np.atleast_2d(psi)
    # Normalize the format of 'ham'
    if isinstance(ham, kwant.system.System):
        ham = ham.hamiltonian_submatrix(params=params, sparse=True)
    try:
        ham = scipy.sparse.csr_matrix(ham)
    except Exception:
        raise ValueError("'ham' is neither a matrix nor a Kwant system.")

    def proj_A(vec):
        return (psi.conj() @ vec.T).T @ psi
    def proj_B(vec):
        return vec - (psi.conj() @ vec.T).T @ psi

    # project the vectors to the subspace `B`
    p_vectors = proj_B((ham @ psi.T).T)
#     # Build the greens functions for these vectors
#     green = build_greens_function(ham, params=params, vectors=p_vectors)
#     psi_i = green(eigenvalues)
#     # we want only the diagonal elements that map each vector to its eigenvalue
#     psi_i = np.diagonal(psi_i, axis1=1, axis2=2)
    """
    This is may be too much of computation, because this evaluates
    the Green's function of each vector to every other energy.
    It could be less expensive to build each green function and evaluate
    it for each energy. Like this
    """
    greens = lambda vec, e: build_greens_function(
        ham, params=params, vectors=proj_B(vec))(e).squeeze()
    psi_i = np.array([greens(vec, e)
                      for (vec, e) in zip(p_vectors, eigenvalues)]).T
    # evaluate for all the energies

    ham_ij = psi_i.conj().T @ psi_i
    return ham_ij

In [486]:
vec = np.atleast_2d(vectors[0])

In [487]:
psi = vectors

In [488]:
vec.shape

(1, 10)

In [489]:
(psi.conj() @ vec.T).T @ psi

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [490]:
vec - (psi.conj() @ vec.T).T @ psi

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [491]:
vec = (ham @ vec.T).T

In [492]:
vec.shape

(1, 10)

In [493]:
build_perturbation(eigenvalues, vectors, ham)

array([[ 132.00528015  +0.j        ,   11.89253877 +89.59757249j,
         -19.84011813-114.69985409j],
       [  11.89253877 -89.59757249j,  614.17399098  +0.j        ,
         -92.10016620  -3.88725425j],
       [ -19.84011813+114.69985409j,  -92.10016620  +3.88725425j,
         351.11349124  +0.j        ]])

In [494]:
psi.shape

(3, 10)

#### apply to a specific realization of $H_0$ and $H$

In [495]:
# from kpm_funcs import build_perturbation

In [496]:
ham0 = np.diag(np.random.rand(10) + 3 + 0j)

In [497]:
np.linalg.norm(ham0)

11.562701178087046

In [498]:
ham = kwant.rmt.gaussian(10) / 50

In [499]:
np.linalg.norm(ham)

0.27362655640194788

In [500]:
eigenvalues0, psi0 = np.linalg.eigh(ham0)

In [501]:
eigenvalues, psi = np.linalg.eigh(ham0+ham)

In [502]:
num = 3
hij = build_perturbation(eigenvalues0[:num], psi0[:,:num].T, ham0)

In [503]:
hij

array([[ 0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j]])

In [504]:
h_eff = ham0[:num,:num] + hij + hij.conj().T

In [505]:
e_eff, psi_eff = np.linalg.eigh(h_eff)

In [506]:
eigenvalues0

array([ 3.2093241 ,  3.43869929,  3.45809193,  3.56738235,  3.67911395,
        3.70810615,  3.74861241,  3.84851607,  3.88625987,  3.95410046])

In [507]:
eigenvalues

array([ 3.17447536,  3.39964186,  3.45143085,  3.57330549,  3.63980566,
        3.68840329,  3.7437198 ,  3.84001982,  3.8584946 ,  3.97446047])

In [508]:
e_eff

array([ 3.2093241 ,  3.45809193,  3.88625987])

In [510]:
g = build_greens_function(ham, vectors=psi0[:,:num].T)

In [516]:
g(eigenvalues0)

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:51: RuntimeWarning: invalid value encountered in arccos
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:52: RuntimeWarning: invalid value encountered in sqrt


array([[[ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj]],

       [[ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj]],

       [[ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
        [ nan+nanj,  nan+nanj,  nan+nanj],
       

In [517]:
eigenvalues0

array([ 3.2093241 ,  3.43869929,  3.45809193,  3.56738235,  3.67911395,
        3.70810615,  3.74861241,  3.84851607,  3.88625987,  3.95410046])